In [1]:
from transform_coordinates import transform_coordinates
from smooth_data import smooth_data
from label_decision import label_decision
from label_trials import label_trials
from append_time_info import append_time_info
import os

In [2]:
video_folder = '/Users/yang/Documents/Wilbrecht_Lab/SLEAP_video'
filename = 'RRM030_Day139_R1_turns.avi.predictions_analysis.h5'

input_h5 = os.path.join(video_folder, filename)
columns_to_smooth = [
        'warped Head x', 'warped Head y',
        'warped Neck x', 'warped Neck y',
        'warped Torso x', 'warped Torso y',
        'warped Tailhead x', 'warped Tailhead y'
    ]

df = transform_coordinates(input_h5)
appended_df = append_time_info(df, video_folder, filename)

smoothed_df = smooth_data(appended_df, columns_to_smooth)

# label decisions
labeled_df = label_decision(smoothed_df)

labeled_t_df = label_trials(labeled_df)


In [4]:
labeled_t_df.head(50)

,Head x,Head y,Neck x,Neck y,Torso x,Torso y,Tailhead x,Tailhead y,warped Head x,warped Head y,...,time,idx,label,rel_time,restaurant,lapIndex,trial,Elapsed Time,decision,trial num
0,94.829071,118.577126,89.464462,118.644859,77.575218,118.163727,68.489502,114.866417,289.887515,128.112782,...,112.561216,3373,tone_onset,0.009280,1.0,1.0,1.0,0.000000,None,1
1,102.386147,118.973137,95.124664,118.973381,85.237984,118.679520,74.103249,117.863655,290.173347,121.087565,...,112.594330,3374,tone_onset,0.042394,1.0,1.0,1.0,0.033333,None,1
2,110.289856,119.163742,102.942169,119.099136,92.868217,118.788300,80.984344,118.277557,290.377800,113.844766,...,112.627725,3375,tone_onset,0.075789,1.0,1.0,1.0,0.066667,None,1
3,118.426064,119.201485,111.062378,119.127686,98.831917,118.799881,89.113869,118.334122,290.520701,106.345987,...,112.661005,3376,tone_onset,0.109069,1.0,1.0,1.0,0.100000,None,1
4,126.872307,119.583473,119.408791,119.394844,107.104393,118.849686,97.819115,118.299698,290.691989,98.681026,...,112.694502,3377,tone_onset,0.142566,1.0,1.0,1.0,0.133333,None,1
5,135.125534,119.684044,127.626724,119.720032,115.419014,119.533012,105.836571,119.148582,290.978013,90.986484,...,112.727885,3378,tone_onset,0.175949,1.0,1.0,1.0,0.166667,None,1
6,143.834991,119.653519,138.570312,121.631302,125.968117,121.891754,113.886505,121.837204,291.651980,83.181843,...,112.761229,3379,NaN,0.209293,1.0,1.0,1.0,0.200000,None,1
7,151.952652,122.043930,146.746521,121.977356,134.554962,122.006775,122.577255,121.969254,292.758947,75.058981,...,112.794586,3380,NaN,0.242650,1.0,1.0,1.0,0.233333,None,1
8,162.694122,122.481537,155.351501,122.322815,143.290619,122.137001,131.355255,121.797050,293.541412,66.276706,...,112.827904,3381,NaN,0.275968,1.0,1.0,1.0,0.266667,None,1
9,170.825729,122.574303,163.401901,122.573166,151.109207,122.686096,139.280487,122.435608,293.842146,58.059998,...,112.860992,3382,T_Entry,0.309056,1.0,1.0,1.0,0.300000,None,1


In [6]:
df = labeled_t_df
set(df['trial'])

{1.0,
 4.0,
 8.0,
 12.0,
 16.0,
 20.0,
 24.0,
 28.0,
 32.0,
 36.0,
 40.0,
 43.0,
 46.0,
 50.0,
 54.0,
 58.0,
 61.0,
 65.0,
 69.0,
 73.0,
 77.0,
 81.0,
 85.0,
 88.0,
 91.0,
 95.0,
 99.0,
 103.0,
 107.0,
 111.0,
 115.0,
 119.0,
 123.0,
 127.0,
 131.0,
 135.0,
 138.0,
 142.0,
 146.0,
 150.0,
 154.0,
 157.0,
 160.0,
 164.0,
 168.0,
 172.0,
 176.0}